In [0]:
%run ./utils

In [0]:
from pyspark.sql  import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,FloatType,DateType
from pyspark.sql.functions import upper, col,round,trim,md5,concat_ws,lit,current_timestamp,to_date

#creating spark session
spark = SparkSession.builder.appName('f1_dwh').getOrCreate()

#creating mount  point 
s3_mnt()

There is already mount point with f1-s3-mnt


In [0]:
driver_schema =  StructType([
    StructField("driverId",IntegerType(),False), \
    StructField("driverRef",StringType(),True), \
    StructField("number",IntegerType(),True), \
    StructField("code", StringType(), True), \
    StructField("forename", StringType(), True), \
    StructField("surname", StringType(), True), \
    StructField("dob", StringType(), True), \
    StructField("nationality", StringType(), True), \
    StructField("url", StringType(), True) \
                        ])
driver_df = spark.read.option('header',True).schema(driver_schema).csv('dbfs:/mnt/f1-s3-mnt/raw_files/drivers.csv')
# driver_df.show()

#cleansing 
for columns in driver_df.columns:
    if driver_df.schema[columns].dataType == StringType()and columns != 'url' :
        driver_df = driver_df.withColumn(columns, trim(upper(col(columns)))).replace("\\N","",subset= [columns])
    elif driver_df.schema[columns].dataType == IntegerType():
        driver_df = driver_df.fillna(-1,subset = [columns])
    driver_df = driver_df.withColumnRenamed(columns, 'd_'+columns.upper())

#changing the DOB columns to datetype
driver_df = driver_df.withColumn('d_DOB',to_date(col('d_DOB'),'yyyy-mm-dd'))
#creating data vault columns  in dataframe like load_date,CIRCUITID_HK,CIRCUITS_HASDIFF
driver_df = driver_df.withColumn('d_DRIVERID_HK',upper(md5(col('d_DRIVERID').cast(StringType()))))
#calculating has diff
driver_df = driver_df.withColumn('d_DRIVER_HASDIFF',upper(md5(concat_ws('',*driver_df.columns[1:])).cast(StringType())))
#adding record_source & Load date
driver_df = driver_df.withColumn('d_RECORD_SOURCE',upper(lit('driver.CSV'))).withColumn('d_LOAD_DATE',current_timestamp())
#verifying the changes
driver_df.printSchema()
driver_df.show(5)

root
 |-- d_DRIVERID: integer (nullable = true)
 |-- d_DRIVERREF: string (nullable = true)
 |-- d_NUMBER: integer (nullable = true)
 |-- d_CODE: string (nullable = true)
 |-- d_FORENAME: string (nullable = true)
 |-- d_SURNAME: string (nullable = true)
 |-- d_DOB: date (nullable = true)
 |-- d_NATIONALITY: string (nullable = true)
 |-- d_URL: string (nullable = true)
 |-- d_DRIVERID_HK: string (nullable = true)
 |-- d_DRIVER_HASDIFF: string (nullable = false)
 |-- d_RECORD_SOURCE: string (nullable = false)
 |-- d_LOAD_DATE: timestamp (nullable = false)

+----------+-----------+--------+------+----------+----------+----------+-------------+--------------------+--------------------+--------------------+---------------+--------------------+
|d_DRIVERID|d_DRIVERREF|d_NUMBER|d_CODE|d_FORENAME| d_SURNAME|     d_DOB|d_NATIONALITY|               d_URL|       d_DRIVERID_HK|    d_DRIVER_HASDIFF|d_RECORD_SOURCE|         d_LOAD_DATE|
+----------+-----------+--------+------+----------+----------+--

LOADING DATA INTO HUB_driver & SAT_driver

In [0]:
from pyspark.sql.functions import broadcast

#adding HUB_drivers in RDV_TABLE/
#checking if there is already hub circuits ,if so then read data from exisitng hub & do left outer join to insert new records

#check if hub_circuits file exists
file_check = {'HUB_DRIVERS':False,'SAT_DRIVERS':False}
try:
        dbutils.fs.ls ("/mnt/f1-s3-mnt/processed_file/RDV_TABLE/HUB_DRIVERS")
        file_check['HUB_DRIVERS'] = True
except Exception as e:
      file_check['HUB_DRIVERS'] = False
        
#check if hub_circuits file exists
try:
        dbutils.fs.ls ("/mnt/f1-s3-mnt/processed_file/RDV_TABLE/SAT_DRIVERS")
        file_check['SAT_DRIVERS'] = True
except Exception as e:
        file_check['SAT_DRIVERS'] =  False
print(file_check)
#adding HUB_circuits in RDV_TABLE/
#checking if there is already hub circuits ,if so then read data from exisitng hub & do left outer join to insert new records
try:
    if file_check['HUB_DRIVERS']:
        hub_df = spark.read.parquet('/mnt/f1-s3-mnt/processed_file/RDV_TABLE/HUB_DRIVERS')  
        temp_hub_df = hub_df.join(broadcast(driver_df.select(col('d_DRIVERID_HK'),col('d_DRIVERID') ,\
                                                           col('d_RECORD_SOURCE'),col('d_LOAD_DATE'))),\
                                                           driver_df.d_DRIVERID_HK == hub_df.DRIVERID_HK,'rightouter').\
                            filter(col('DRIVERID_HK').isNull()).\
                            select(col('d_DRIVERID_HK'),col('d_DRIVERID'),col('d_RECORD_SOURCE'),col('d_LOAD_DATE'))
        for columns in temp_hub_df.columns:
             temp_hub_df = temp_hub_df.withColumnRenamed(columns, columns[2:])
        #verifying  New Schema 
        temp_hub_df.printSchema()
        temp_hub_df.show()
        #apending new df DATA to HUB_CIRCUITS
        temp_hub_df.write.mode('append').format('parquet').save('/mnt/f1-s3-mnt/processed_file/RDV_TABLE/HUB_DRIVERS')
    else:
        print('May be file is missing or data being loaded freshly in HUB_driver')
        temp_df = driver_df
        for columns in temp_df.columns:
            temp_df = temp_df.withColumnRenamed(columns, columns[2:])
        temp_df.printSchema()
        #write the file for if there is no hub file
        temp_df.select(col('DRIVERID_HK'),col('DRIVERID'),col('RECORD_SOURCE'),col('LOAD_DATE')).write.mode('overwrite').format('parquet').save('/mnt/f1-s3-mnt/processed_file/RDV_TABLE/HUB_DRIVERS')
except Exception as e:
    print(e)
# adding SAT_CIRCUITS in RDV_TABLE/
try:
    if file_check['SAT_DRIVERS']:
        sat_df = spark.read.parquet('/mnt/f1-s3-mnt/processed_file/RDV_TABLE/SAT_DRIVERS')
        temp_sat_df =  sat_df.join(broadcast(driver_df.select( col('d_DRIVERREF'),col('d_NUMBER'),col('d_CODE'),col('d_FORENAME'), col('d_SURNAME'), \
                                                            col('d_DOB'), col('d_NATIONALITY'),col('d_URL'),col('d_DRIVERID_HK'), \
                                                            col('d_RECORD_SOURCE'), col('d_LOAD_DATE'),col('d_DRIVER_HASDIFF'))),sat_df.DRIVERID_HK==driver_df.d_DRIVERID_HK,'rightouter').\
                                                            filter(col('DRIVERID_HK').isNull()).\
                                                                select(col('d_RECORD_SOURCE'), col('d_LOAD_DATE'),col('d_DRIVERID_HK'),\
                                                                       col('d_DRIVER_HASDIFF'), col('d_DRIVERREF'), col('d_NUMBER'), col('d_CODE'),\
                                                                       col('d_FORENAME'), col('d_SURNAME'), col('d_DOB'), col('d_NATIONALITY'), col('d_URL'))
        for columns in temp_sat_df.columns:
            temp_sat_df = temp_sat_df.withColumnRenamed(columns, columns[2:])
        #verifying  New Schema
        temp_sat_df.printSchema()
        temp_sat_df.show()
        #adding computed file to sat_circuits table
        temp_df.write.mode('append').format('parquet').save('/mnt/f1-s3-mnt/processed_file/RDV_TABLE/SAT_DRIVERS')
    else:
        print('May be file is missing or data being loaded freshly')
        temp_df = driver_df
        for columns in temp_df.columns:
            temp_df = temp_df.withColumnRenamed(columns, columns[2:])
        temp_df.printSchema()
        temp_df.write.mode('overwrite').format('parquet').save('/mnt/f1-s3-mnt/processed_file/RDV_TABLE/SAT_DRIVERS')
except  Exception as e:
    print(e)
  


{'HUB_DRIVERS': True, 'SAT_DRIVERS': True}
root
 |-- DRIVERID_HK: string (nullable = true)
 |-- DRIVERID: integer (nullable = true)
 |-- RECORD_SOURCE: string (nullable = false)
 |-- LOAD_DATE: timestamp (nullable = false)

+-----------+--------+-------------+---------+
|DRIVERID_HK|DRIVERID|RECORD_SOURCE|LOAD_DATE|
+-----------+--------+-------------+---------+
+-----------+--------+-------------+---------+

root
 |-- RECORD_SOURCE: string (nullable = false)
 |-- LOAD_DATE: timestamp (nullable = false)
 |-- DRIVERID_HK: string (nullable = true)
 |-- DRIVER_HASDIFF: string (nullable = false)
 |-- DRIVERREF: string (nullable = true)
 |-- NUMBER: integer (nullable = true)
 |-- CODE: string (nullable = true)
 |-- FORENAME: string (nullable = true)
 |-- SURNAME: string (nullable = true)
 |-- DOB: date (nullable = true)
 |-- NATIONALITY: string (nullable = true)
 |-- URL: string (nullable = true)

+-------------+---------+-----------+--------------+---------+------+----+--------+-------+---

In [0]:
dbutils.fs.ls ("/mnt/f1-s3-mnt/processed_file/RDV_TABLE/HUB_DRIVERS")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-120387050022155> in <module>
----> 1 dbutils.fs.ls ("/mnt/f1-s3-mnt/processed_file/RDV_TABLE/HUB_DRIVERS")

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    379                     exc.__context__ = None
    380                     exc.__cause__ = None
--> 381                     raise exc
    382 
    383             return f_with_exception_handling

ExecutionError: An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: /mnt/f1-s3-mnt/processed_file/RDV_TABLE/HUB_DRIVERS
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:120)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listSt